## Вводная часть
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней:
- информация о книгах,
- издательствах,
- авторах, 
- пользовательские оценки книг,
- пользовательские обзоры книг. 

## Цель исследования:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Описание таблиц

In [2]:
# таблица books
query='''select *
from books
'''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


### Таблица books
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

In [3]:
# таблица authors
query='''select *
from authors
'''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


### Таблица authors
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

In [4]:
# таблица publishers
query='''select *
from publishers
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


### Таблица publishers
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

In [5]:
# таблица ratings
query='''select *
from ratings
'''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


### Таблица ratings
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

In [6]:
# таблица reviews
query='''select *
from reviews
'''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...



### Таблица reviews
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Исследование данных

### Посчитаем, сколько книг вышло после 1 января 2000 года;

In [7]:
# таблица books
query='''
select count(title)
from books
where publication_date  > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


Вывод поле 2000-01-01 вышла 821 книга

### Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [8]:
query='''
select b.book_id, b.title, count(distinct review_id), round(avg(rating),2)
from books as b
full outer join reviews as re on b.book_id=re.book_id
join ratings as ra on b.book_id=ra.book_id
group by b.book_id
order by 3 desc,4 desc 
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count,round
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Вывод: для каждой книги нашли количество обзоров и среднюю оценку

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [9]:
query='''
select publisher, count(b.book_id)
from books as b
full outer join publishers as p on b.publisher_id=p.publisher_id
where num_pages > 50
group by publisher
order by 2 desc
limit 1
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher,count
0,Penguin Books,42


Вывод: Penguin Books выпустило 42 книги

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:
query='''
select author, round(avg(ra.rating),3)
from authors as a
full outer join books as b on a.author_id=b.author_id
join ratings as ra on b.book_id=ra.book_id
where b.book_id in
     (select book_id
     from ratings as ra
     group by book_id
     having count(rating) >=50)
group by author
order by 2 desc
limit 1
     
'''
pd.io.sql.read_sql(query, con = engine)

,author,round
0,J.K. Rowling/Mary GrandPré,4.287


Вывод: автор с самой высокой оценкой J.K. Rowling/Mary GrandPré, рейтинг 4.287

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
query='''
select avg(count)
from
   (select username,  count(review_id) as count
    from reviews
    where username in 
         (select username
          from ratings
          group by username
          having count(rating_id) > 50)
          group by username) as t



'''
pd.io.sql.read_sql(query, con = engine) 

,avg
0,24.333333


Вывод: среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24,3

## Обший вывод: 

Мы помотрели базу данных, какие входят таблицы и значения каждого столбца. 
Ответили на вопросы задания:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.